In [1]:
import numpy as np
import pandas as pd
import scipy as sp
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
import geopandas as gpd
from shapely.geometry import Point

In [3]:
data = pd.read_csv('Data/kody_stacji.csv', encoding='windows-1250',delimiter=";")    
#Zwykłe kodowanie utf-8 nie działało --> po otwarciu pliku w Excelu można sprawdzić rodzaj kodowania 
data = data.set_index('LP.')   #ustawiam indeks 
data

,ID,Nazwa,Rzeka,Szerokość geograficzna,Długość geograficzna,Wysokość n.p.m.
LP.,,,,,,
1,250180590,RYBNIK-STODOŁY,Ruda,50 09 16,18 28 59,216
2,254140010,DZIWNÓW,NaN,54 01 20,14 43 54,7
3,250160530,ZIELENIEC,Bystrzyca Dusznicka,50 19 48,16 23 39,840
4,250160630,MIĘDZYGÓRZE,Wilczka,50 13 06,16 46 23,800
5,250160650,MIĘDZYLESIE,Nysa Kłodzka,50 09 12,16 40 15,453
...,...,...,...,...,...,...
637,250140020,SIENIAWKA,Nysa Łużycka,50 53 38,14 50 08,234
638,250160920,PIŁAWA GÓRNA,Piława,50 41 22,16 43 57,295
639,249180590,RUPTAWA,Olza,49 56 12,18 35 22,230
